# **NYC High Volume FHV Data Analysis - Companies: Uber, Lyft, Juno and Via**

Exploratory analysis on NYC (e.g., Uber, Lyft, Juno and Via) looking into customer segmentation and factors that may impact tipping behaviors across the 5 boroughs

--- 

### **Imports & Setup**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../scripts') # imports custom scripts from the scripts directory
from data_utils import remove_outliers

# setting display options for pandas
pd.set_option('display.max_columns', None) # displays all columns in the dataframe
pd.set_option('display.max_rows', 100) # sets the max number of rows to 100

### **Loading Data**

In [4]:
# look up tables
zone = pd.read_csv('../data/taxi_zone_lookup.csv') 
vendor = pd.read_csv('../data/taxi_vendor_lookup.csv')
payment = pd.read_csv('../data/payment_lookup.csv')
hvfhs = pd.read_csv('../data/hvfhs_lookup.csv')

# fhvhv data
df = pd.read_parquet('../data/fhvhv_tripdata_2025-05.parquet')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21091193 entries, 0 to 21091192
Data columns (total 25 columns):
 #   Column                Dtype         
---  ------                -----         
 0   hvfhs_license_num     object        
 1   dispatching_base_num  object        
 2   originating_base_num  object        
 3   request_datetime      datetime64[us]
 4   on_scene_datetime     datetime64[us]
 5   pickup_datetime       datetime64[us]
 6   dropoff_datetime      datetime64[us]
 7   PULocationID          int32         
 8   DOLocationID          int32         
 9   trip_miles            float64       
 10  trip_time             int64         
 11  base_passenger_fare   float64       
 12  tolls                 float64       
 13  bcf                   float64       
 14  sales_tax             float64       
 15  congestion_surcharge  float64       
 16  airport_fee           float64       
 17  tips                  float64       
 18  driver_pay            float64       
 19

In [11]:
df.describe()

,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,cbd_congestion_fee
count,21091193,21091193,21091193,21091193,2.109119e+07,2.109119e+07,2.109119e+07,2.109119e+07,2.109119e+07,2.109119e+07,2.109119e+07,2.109119e+07,2.109119e+07,2.109119e+07,2.109119e+07,2.109119e+07,2.109119e+07
mean,2025-05-16 09:17:08.694439,2025-05-16 09:21:11.248639,2025-05-16 09:22:05.263951,2025-05-16 09:42:49.797092,1.384395e+02,1.425403e+02,5.097999e+00,1.244524e+03,2.809792e+01,1.116688e+00,6.993011e-01,2.328690e+00,9.926219e-01,2.244738e-01,1.240683e+00,2.169880e+01,5.148894e-01
min,2025-04-30 23:36:17,2025-04-30 23:34:18,2025-05-01 00:00:00,2025-05-01 00:03:01,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-5.096000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.712000e+01,0.000000e+00
25%,2025-05-08 17:49:55,2025-05-08 17:54:24,2025-05-08 17:55:17,2025-05-08 18:20:32,7.500000e+01,7.600000e+01,1.550000e+00,6.080000e+02,1.264000e+01,0.000000e+00,3.100000e-01,1.040000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.490000e+00,0.000000e+00
50%,2025-05-16 09:10:28,2025-05-16 09:13:27,2025-05-16 09:14:27,2025-05-16 09:34:32,1.380000e+02,1.410000e+02,3.020000e+00,9.960000e+02,1.999000e+01,0.000000e+00,4.900000e-01,1.680000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.612000e+01,0.000000e+00
75%,2025-05-23 22:19:23,2025-05-23 22:23:19,2025-05-23 22:24:13,2025-05-23 22:43:07,2.090000e+02,2.160000e+02,6.390000e+00,1.598000e+03,3.350000e+01,0.000000e+00,8.300000e-01,2.860000e+00,2.750000e+00,0.000000e+00,0.000000e+00,2.738000e+01,1.500000e+00
max,2025-06-01 00:16:47,2025-06-01 00:02:41,2025-05-31 23:59:59,2025-06-01 03:19:25,2.650000e+02,2.650000e+02,2.761800e+02,3.503600e+04,1.691790e+03,9.349000e+01,4.228000e+01,1.584500e+02,5.500000e+00,7.500000e+00,1.994000e+02,9.542100e+02,3.000000e+00
std,NaN,NaN,NaN,NaN,7.482419e+01,7.804860e+01,5.975552e+00,9.263445e+02,2.604773e+01,3.650176e+00,6.689067e-01,2.130185e+00,1.316590e+00,7.185373e-01,3.788560e+00,1.898420e+01,7.121967e-01


In [7]:
df.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,cbd_congestion_fee
0,HV0003,B03404,B03404,2025-04-30 23:59:52,2025-05-01 00:03:02,2025-05-01 00:03:25,2025-05-01 00:12:22,160,82,2.51,537,12.54,0.0,0.31,1.13,0.00,0.0,0.0,10.47,N,N,N,N,N,0.0
1,HV0003,B03404,B03404,2025-05-01 00:05:01,2025-05-01 00:06:30,2025-05-01 00:07:04,2025-05-01 00:17:17,82,129,1.51,614,11.14,0.0,0.21,0.76,0.00,0.0,0.0,8.34,N,N,N,N,N,0.0
2,HV0003,B03404,B03404,2025-05-01 00:16:00,2025-05-01 00:19:06,2025-05-01 00:20:32,2025-05-01 00:39:01,129,226,5.01,1109,18.82,0.0,0.47,1.66,0.00,0.0,0.0,18.28,N,N,N,N,N,0.0
3,HV0005,B03406,None,2025-05-01 00:44:56,2025-05-01 00:49:38,2025-05-01 00:50:38,2025-05-01 01:00:23,37,112,1.29,585,10.73,0.0,0.27,0.95,0.00,0.0,0.0,8.26,N,N,N,N,Y,0.0
4,HV0003,B03404,B03404,2025-05-01 00:04:08,2025-05-01 00:04:39,2025-05-01 00:06:24,2025-05-01 00:31:44,97,87,5.04,1520,34.21,0.0,0.82,3.04,2.75,0.0,0.0,22.44,N,N,N,N,N,1.5


---
### **Data Assumptions**
* Each row represents a single trip record
* No explicit primary key column
* Each row's uniqueness could be composed of a combination of columns (pickup, dropoff, vendor) 
  
*Note: for this exploratory analysis, a primary key is not critical, as much of the analysis is focused on aggregation patterns rather than uniquely identifying rows* 
> If necessary, could implement 'ride_id' as a primary key using row index 

---
### **Data Cleaning**

In [6]:
df.isna().sum()

hvfhs_license_num             0
dispatching_base_num          0
originating_base_num    6034550
request_datetime              0
on_scene_datetime             0
pickup_datetime               0
dropoff_datetime              0
PULocationID                  0
DOLocationID                  0
trip_miles                    0
trip_time                     0
base_passenger_fare           0
tolls                         0
bcf                           0
sales_tax                     0
congestion_surcharge          0
airport_fee                   0
tips                          0
driver_pay                    0
shared_request_flag           0
shared_match_flag             0
access_a_ride_flag            0
wav_request_flag              0
wav_match_flag                0
cbd_congestion_fee            0
dtype: int64

In [8]:
df['originating_base_num'].unique()

array(['B03404', None, 'B03406', 'B02026', 'B02003'], dtype=object)

In [9]:
# also replace np.nan with NA to standardize
df['originating_base_num'].replace({None: pd.NA}, inplace=True)

In [10]:
#checking for duplicates
df.duplicated().sum()

0

---
### **Exploratory**: The 5 boroughs of NYC